In [2]:
from pyomo.environ import * 
from pyomo.opt import SolverFactory
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def lagrangian_branch_and_cut(scenario_models, initial_multipliers, tolerance=1e-3):
    N = len(scenario_models)
    
    # 1. 初始化
    global_model = ConcreteModel()
    global_model.N = RangeSet(1, N)
    
    # 2. 初始化 linking variables: x, y
    # 注意：每个 scenario 都有自己的 copy
    global_model.x = Var(global_model.N, within=Reals, bounds=(0, 10))  # 例子
    global_model.y = Var(global_model.N, within=Binary)

    # 3. 加入每个 scenario 的 local 模型（已经构造好）
    global_model.submodels = Block(global_model.N)
    for n in global_model.N:
        global_model.submodels[n].transfer_attributes_from(scenario_models[n-1])

    # 4. 加入耦合约束（等价于 x1 = x2 = ... = xN）
    def coupling_x_rule(m, n):
        if n < N:
            return global_model.x[n] == global_model.x[n+1]
        return Constraint.Skip
    global_model.couple_x = Constraint(global_model.N, rule=coupling_x_rule)

    def coupling_y_rule(m, n):
        if n < N:
            return global_model.y[n] == global_model.y[n+1]
        return Constraint.Skip
    global_model.couple_y = Constraint(global_model.N, rule=coupling_y_rule)

    # 5. 添加拉格朗日松弛目标函数（先作为示例构造）
    global_model.obj = Objective(expr=sum(
        1/N * scenario_objective(n, global_model.submodels[n], global_model.x[n], global_model.y[n], initial_multipliers, n)
        for n in global_model.N
    ), sense=minimize)

    return global_model


In [4]:
def update_lagrange_multipliers(x_vals, y_vals, multipliers, UB, LB, beta=0.5):
    N = len(x_vals)
    lambda_x_new = []
    lambda_y_new = []

    # 计算次梯度范数
    numerator = UB - LB
    denom = 0.0
    for n in range(N - 1):
        denom += (x_vals[n] - x_vals[n+1])**2
        denom += (y_vals[n] - y_vals[n+1])**2

    if denom == 0:
        step_size = 0
    else:
        step_size = beta * numerator / denom

    for n in range(N - 1):
        new_lx = multipliers['lambda_x'][n] + step_size * (x_vals[n] - x_vals[n+1])
        new_ly = multipliers['lambda_y'][n] + step_size * (y_vals[n] - y_vals[n+1])
        lambda_x_new.append(new_lx)
        lambda_y_new.append(new_ly)

    return {
        'lambda_x': lambda_x_new,
        'lambda_y': lambda_y_new
    }


In [5]:
def update_lagrange_multipliers(
    h_val: np.ndarray,
    lambda_prev: np.ndarray,
    UB: float,
    LB: float,
    gamma: float = 2.0,
    project_nonnegative: bool = True
) -> np.ndarray:
    """
    Update the Lagrange multipliers using the subgradient method
    as described in the appendix of the paper.

    Parameters:
        h_val (np.ndarray): the coupling constraint residual vector h(x^k)
        lambda_prev (np.ndarray): the previous lagrange multiplier λ^k
        UB (float): current upper bound (feasible primal cost)
        LB (float): current lower bound (lagrangian relaxation cost)
        gamma (float): step size coefficient (recommended 1 ~ 5)
        project_nonnegative (bool): whether to project result to non-negative space

    Returns:
        lambda_next (np.ndarray): updated lagrange multiplier
    """
    norm_h2 = np.dot(h_val, h_val)  # ||h(x^k)||^2

    if norm_h2 < 1e-8:  # Avoid division by zero
        return lambda_prev.copy()

    step_size = gamma * (UB - LB) / norm_h2
    lambda_next = lambda_prev + step_size * h_val

    if project_nonnegative:
        lambda_next = np.maximum(lambda_next, 0.0)

    return lambda_next


In [ ]:
def lagrangian_branch_and_cut(scenario_models, initial_multipliers, UB=10, max_iter=10, tolerance=1e-3):
    N = len(scenario_models)
    lambdas_x = initial_multipliers['lambda_x']
    lambdas_y = initial_multipliers['lambda_y']

    def scenario_objective(n, submodel, x_n, y_n, lambdas_x, lambdas_y):
        # Lagrangean penalty terms
        lx = lambdas_x[n - 1] if n <= N - 1 else 0
        lx_prev = lambdas_x[n - 2] if n > 1 else 0
        ly = lambdas_y[n - 1] if n <= N - 1 else 0
        ly_prev = lambdas_y[n - 2] if n > 1 else 0
        weight = 1 / N
        return weight * submodel.obj_base + (lx - lx_prev) * x_n + (ly - ly_prev) * y_n

    for _ in range(max_iter):
        model = pyo.ConcreteModel()
        model.N = pyo.RangeSet(1, N)
        model.x = pyo.Var(model.N, within=pyo.Reals, bounds=(0, 10))
        model.y = pyo.Var(model.N, within=pyo.Binary)
        model.submodels = pyo.Block(model.N)

        for n in model.N:
            model.submodels[n].transfer_attributes_from(scenario_models[n-1])

        def coupling_x_rule(m, n):
            if n < N:
                return m.x[n] == m.x[n+1]
            return pyo.Constraint.Skip
        model.couple_x = pyo.Constraint(model.N, rule=coupling_x_rule)

        def coupling_y_rule(m, n):
            if n < N:
                return m.y[n] == m.y[n+1]
            return pyo.Constraint.Skip
        model.couple_y = pyo.Constraint(model.N, rule=coupling_y_rule)

        model.obj = pyo.Objective(expr=sum(
            scenario_objective(n, model.submodels[n], model.x[n], model.y[n], lambdas_x, lambdas_y)
            for n in model.N
        ), sense=pyo.minimize)

        solver = pyo.SolverFactory('glpk')
        solver.solve(model)

        # get x_n, y_n values from solution
        x_vals = [pyo.value(model.x[n]) for n in model.N]
        y_vals = [pyo.value(model.y[n]) for n in model.N]
        LB = pyo.value(model.obj)

        # update lambda using subgradient method
        denom = sum((x_vals[n] - x_vals[n+1])**2 + (y_vals[n] - y_vals[n+1])**2 for n in range(N - 1))
        step = 0.5 * (UB - LB) / denom if denom > 1e-8 else 0.0
        lambdas_x = [lambdas_x[n] + step * (x_vals[n] - x_vals[n+1]) for n in range(N - 1)]
        lambdas_y = [lambdas_y[n] + step * (y_vals[n] - y_vals[n+1]) for n in range(N - 1)]

        # convergence check
        gap = (UB - LB) / UB if UB != 0 else float('inf')
        if gap <= tolerance:
            break

    return LB
